In [ ]:
!pip install numpy
!pip install torch


In [1]:
import vame

from glob import glob
import os.path as osp

Using CUDA
GPU active: True
GPU used: NVIDIA GeForce RTX 3060


In [2]:
from importlib import reload
reload(vame)

<module 'vame' from 'D:\\Project- Electro\\VAME\\vame\\__init__.py'>

In [3]:
# These paths have to be set manually 

project='Unsupervised Learning Tutorial with VAME 2'
# videos = glob(osp.join(r'D:\Project- Electro\VAME\materials\videos','*.avi'))
videos = ''
working_directory = r'D:\Project- Electro\VAME\materials\working_dir_5'
    

## Step 1.1: Initiazile a VAME Project
With the variables above, initialize a new project and save the path to the config.yaml file as config. This will create a new project folder with a predefined structure in your working_directory.

This should create four folders inside your working_directory folder:
- path\to\working\dir\videos:
    - The video files will be copied to this folder, the videos are used for visualisation and debugging. 
    - it is not essential to have any video files, and in fact it slows down some of the processes, however, for your first usage, you should use video files to make sure that everything is in order. 
- path\to\working\dir\data
    - The data folder stores generated data that will be created during pre-processing, when converting the data from csv files into numpy files 
- path\to\working\dir\results
- path\to\working\dir\model
    - The results folder stores everything related to the model, including:
        - best_model : 
        - model_losses
        - pretrained_model   


In [4]:
# config = vame.init_new_project(project=project, videos=videos, working_directory=working_directory, videotype='.avi', num_features=13)

In [5]:
# config = r"D:\Project- Electro\VAME\materials\working_dir\Unsupervised Learning Tutorial with VAME-May28-2024\config.yaml"
# config = r"D:\Project- Electro\VAME\materials\working_dir_2\Unsupervised Learning Tutorial with VAME-Jun30-2024\config.yaml"
config = r"D:\Project- Electro\VAME\materials\working_dir_5\Unsupervised Learning Tutorial with VAME 2-Sep12-2024\config.yaml"


## Step 1.2: Get your data ready

First, move your DeepLabCut .csv data to the corresponding videos/pose_estimation/ directory.

Then, use the function below to align your behavior videos egocentrically given pose_ref_index as a list of reference coordinate for alignment. In this case [0,5]. Example: 0: snout, 1: forehand_left, 2: forehand_right, 3: hindleft, 4: hindright, 5: tail

You don't have to follow these specific feature names, the important thing is that all the columns in all your csv files contain the same features in the same order. After that, select two anchor point that can be used to align your data.
How to choose the anchor point:
- They have to be easily detected and not obscured by other body part
- The distance between the two points should be constant, for example, you can choose the nose tip and the trunk because the distance may change when the animal shifts their head.
- If you are going to use [0,5], make sure that you are accessing the correct features. 

In [7]:
vame.egocentric_alignment(config, pose_ref_index=[0,1])

I am here: align_egocentrical.py
Here are the files: ['supervised_data_male']
Aligning data supervised_data_male, Pose confidence value: 0.99
I am here
dataFile: 
['D:\\Project- Electro\\VAME\\materials\\working_dir_5\\Unsupervised Learning Tutorial with VAME 2-Sep12-2024\\videos\\pose_estimation\\supervised_data_male.csv']


Align frames: 100%|██████████| 240000/240000 [10:49<00:00, 369.36it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Project- Electro\\VAME\\materials\\working_dir_5\\Unsupervised Learning Tutorial with VAME 2-Sep12-2024\\data\\supervised_data_male\\supervised_data_male-PE-seq.npy'

# Step 1.3: Create the Training Set

In [20]:
vame.create_trainset(config)

Creating training dataset...
Using robust setting to eliminate outliers! IQR factor: 4
Creating trainset from the vame.csv_to_numpy() output 


ValueError: need at least one array to concatenate

# Step 2: Train your VAME Model
 
- If you encounter an error stating that the number of features is different than the expected number, 
    - Check the config.yaml file
    - Under "# RNN model general hyperparameter:" -> num_features: 16 
    - Make sure that this number matches what you have in the pose files
 

In [21]:
vame.train_model(config)

Train Variational Autoencoder - model name: VAME 

Using CUDA
GPU active: True
GPU used:  NVIDIA GeForce RTX 3060
Latent Dimensions: 30, Time window: 30, Batch Size: 256, Beta: 1, lr: 0.0005

Initialize train data. Datapoints 26429
Initialize test data. Datapoints 2936
Scheduler step size: 100, Scheduler gamma: 0.20

Start training... 
Epoch: 1
Train loss: 50729.484, MSE-Loss: 31178.844, MSE-Future-Loss 19550.640, KL-Loss: 0.000, Kmeans-Loss: 0.000, weight: 0.00
Test loss: 6864.085, MSE-Loss: 6864.085, KL-Loss: 0.000, Kmeans-Loss: 0.000


Epoch: 2
Train loss: 21576.556, MSE-Loss: 12320.990, MSE-Future-Loss 9255.566, KL-Loss: 0.000, Kmeans-Loss: 0.000, weight: 0.00
Test loss: 3763.600, MSE-Loss: 3763.600, KL-Loss: 0.000, Kmeans-Loss: 0.000


Epoch: 3
Train loss: 14591.747, MSE-Loss: 7222.750, MSE-Future-Loss 7367.271, KL-Loss: 1.174, Kmeans-Loss: 0.551, weight: 0.25
Test loss: 3149.380, MSE-Loss: 3147.465, KL-Loss: 1.347, Kmeans-Loss: 0.569


Epoch: 4
Train loss: 12728.432, MSE-Loss: 59

# Step 3: Evaluate the model

In [5]:
vame.evaluate_model(config)

Using CUDA
GPU active: True
GPU used: NVIDIA GeForce RTX 3060


Evaluation of VAME model. 
Initialize test data. Datapoints 2936
You can find the results of the evaluation in '/Your-VAME-Project-Apr30-2020/model/evaluate/' 
OPTIONS:
- vame.pose_segmentation() to identify behavioral motifs.
- re-run the model for further fine tuning. Check again with vame.evaluate_model()


# Step 4: Motif segmentation

In [23]:
vame.pose_segmentation(config)

Pose segmentation for VAME model: VAME 

Using CUDA
GPU active: True
GPU used: NVIDIA GeForce RTX 3060
Embedding of latent vector for file cropped_Rat1 probe 4-day2-free_2019-05-13-114301-0000


100%|██████████| 9856/9856 [00:13<00:00, 741.71it/s]


Embedding of latent vector for file cropped_Rat1 probe4-day1-free_2019-05-12-104236-0000


100%|██████████| 9936/9936 [00:13<00:00, 743.39it/s]


Embedding of latent vector for file cropped_Rat2-probe1-sniffing-day1-Free_2019-07-14-155327-0000


100%|██████████| 9483/9483 [00:12<00:00, 746.18it/s]


For all animals the same parameterization of latent vectors is applied for 15 cluster
Using a HMM as parameterization!
D:\Project- Electro\VAME\materials\working_dir_2\Unsupervised Learning Tutorial with VAME-Jun30-2024\results\cropped_Rat1 probe 4-day2-free_2019-05-13-114301-0000\VAME\hmm-15\
D:\Project- Electro\VAME\materials\working_dir_2\Unsupervised Learning Tutorial with VAME-Jun30-2024\results\cropped_Rat1 probe4-day1-free_2019-05-12-104236-0000\VAME\hmm-15\
D:\Project- Electro\VAME\materials\working_dir_2\Unsupervised Learning Tutorial with VAME-Jun30-2024\results\cropped_Rat2-probe1-sniffing-day1-Free_2019-07-14-155327-0000\VAME\hmm-15\
You succesfully extracted motifs with VAME! From here, you can proceed running vame.motif_videos() 


# Step 5: Create Motif Videos

In [24]:
vame.motif_videos(config, videoType='.avi')

Cluster size is: 15 
Motif videos getting created for cropped_Rat1 probe 4-day2-free_2019-05-13-114301-0000 ...
fps: 30.0
Cluster: 0


100%|██████████| 252/252 [00:11<00:00, 22.57it/s]


Cluster: 1


100%|██████████| 256/256 [00:11<00:00, 22.70it/s]


Cluster: 2


...

Cluster: 13


100%|██████████| 261/261 [00:13<00:00, 19.49it/s]


Cluster: 14


100%|██████████| 424/424 [00:21<00:00, 19.57it/s]

All videos have been created!


# Step 6: Create behavioral hierarchies

In [6]:
vame.community(config, show_umap=True, cut_tree=2)

D:\Project- Electro\VAME\vame\analysis\community_analysis.py:57: RuntimeWarning: invalid value encountered in true_divide
  transition_matrix = adjacency_matrix/row_sum[:,np.newaxis]


UMAP calculation for file cropped_Rat1 probe 4-day2-free_2019-05-13-114301-0000
Embedding 9856 data points..
Embedding 9856 data points..
UMAP calculation for file cropped_Rat1 probe4-day1-free_2019-05-12-104236-0000
Embedding 9936 data points..
Embedding 9936 data points..
UMAP calculation for file cropped_Rat2-probe1-sniffing-day1-Free_2019-07-14-155327-0000
Embedding 9483 data points..
Embedding 9483 data points..


# Step 7: Create community videos

In [26]:
vame.community_videos(config, videoType='.avi')

Cluster size is: 15 
Community videos getting created for cropped_Rat1 probe 4-day2-free_2019-05-13-114301-0000 ...
fps: 30.0
Cluster: 0


100%|██████████| 1000/1000 [00:45<00:00, 21.86it/s]


Cluster: 1


0it [00:00, ?it/s]


Cluster: 2


...

Cluster: 13


0it [00:00, ?it/s]


Cluster: 14


0it [00:00, ?it/s]

All videos have been created!


# Step 8: Create UMAP visualization

In [5]:
vame.visualization(config, label=None) #options: label: None, "motif", "community"

Visualizing 9856 data points.. 
Visualizing 9936 data points.. 
Visualizing 9483 data points.. 


# Step 9: Create GIF Animation

In [6]:
# Note: This function is currently very slow. Once the frames are saved you can create a video
# or gif via e.g. ImageJ or other tools
# vame.gif(config, pose_ref_index=[0,5], subtract_background=True, start=None, 
#          length=500, max_lag=30, label='community', file_format='.mp4', crop_size=(300,300))

vame.gif(config, pose_ref_index=[4,5], subtract_background=True, start=None, 
         length=500, max_lag=30, label='community', file_format='.avi', crop_size=(300,300))

Loading background image ...


100%|██████████| 500/500 [14:34<00:00,  1.75s/it]


Loading background image ...


100%|██████████| 500/500 [14:43<00:00,  1.77s/it]


Loading background image ...


100%|██████████| 500/500 [14:48<00:00,  1.78s/it]


# Step 10: Create a generative model

In [4]:
vame.generative_model(config, mode="centers") #options: mode: "sampling", "reconstruction", "centers", "motifs"

Load model... 


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Project- Electro\\VAME\\materials\\working_dir_2\\Unsupervised Learning Tutorial with VAME-Jun30-2024\\results\\cropped_Rat1 probe 4-day2-free_2019-05-13-114301-0000\\VAME\\kmeans-15\\cluster_center_cropped_Rat1 probe 4-day2-free_2019-05-13-114301-0000.npy'